In [1]:
import os
import math
import numpy as np
import pandas as pd
import pywt
import matplotlib.pyplot as plt
from scipy.signal import argrelextrema, find_peaks
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)
import cv2
import glob
import scipy.stats as stats

In [2]:
import random
mu = 0
sigma = 0.12
# mu : mean
# sigma : standard deviation

In [3]:
#def wavelet_detection_synthetic_trajectory():
def time_acc_dec_jerk_def():
    # state_change_label: 0 for A-F, 1 for A-D, 2 for S-A, 3 for F-D, 4 for D-S
    total_time_step = 200
    time_resolution = 0.1
    
    acceleration_range = [1, 3]
    acc_resolution = 0.5
    # (3-1)/0.5 + 1 = 5
    total_num_acc = int((acceleration_range[1] - acceleration_range[0]) / acc_resolution) + 1
    
    deceleration_range = [-3, -1]
    dec_resolution = 0.5
    total_num_dec = int((deceleration_range[1] - deceleration_range[0]) / dec_resolution) + 1
    
    # absolute_jerk_value = 2  # absolute value
    jerk_value_range = [0.5, 1, 1.5, 2, 2.5]  # absolute value
    change_point_range = [8, 12]
    change_point_resolution = 0.5
    # (12-8)/0.5 = 8
    total_num_change_point = int((change_point_range[1] - change_point_range[0]) / change_point_resolution)
    
    return total_time_step, time_resolution, acceleration_range, acc_resolution, total_num_acc, deceleration_range, dec_resolution, total_num_dec, jerk_value_range, change_point_range, change_point_resolution, total_num_change_point

In [4]:
def A_F_trajectory_generation():
    global all_state_change_point_record
    global all_synthetic_trj
    
    current_state_change_label = 0
    current_starting_speed = 0
    current_veh_id = 0
    
    total_time_step, time_resolution, acceleration_range, acc_resolution, total_num_acc, deceleration_range, dec_resolution, total_num_dec, jerk_value_range, change_point_range, change_point_resolution, total_num_change_point = time_acc_dec_jerk_def()
    
    # i in {0, 1, ... 7}
    for i in range(total_num_change_point):
        # {8, 8.5, 9, ...11.5}
        current_change_point_time = change_point_range[0] + i * change_point_resolution
        # {0, 1, ... 4}
        for j in range(total_num_acc):
            # {1, 1.5, 2, ... 3}
            current_acc = acceleration_range[0] + acc_resolution * j
            # {0, 1, ... 4}
            for k in range(len(jerk_value_range)):
                current_absolute_jerk = jerk_value_range[k]
                
                # print some info
                print_str = 'Current generating ' + str(current_veh_id) + 'th vehicle synthetic trajectory '
                print(print_str)
                
                # init data space
                single_scenario_trj = pd.DataFrame(np.zeros((total_time_step, 6)), 
                                                   columns=['veh_id', 'local_time', 'position', 
                                                            'speed', 'acceleration', 'state_change_label'])
                single_change_point_record = pd.DataFrame(np.zeros((1, 9)), 
                                                          columns=['veh_id', 'test_acc', 'test_dec', 'test_jerk', 
                                                                   'change_point_time_1', 'change_point_time_2', 
                                                                   'zero_acc_time', 'mid_change_point_time', 
                                                                   'state_change_label'])
                
                single_change_point_record.loc[:, 'state_change_label'] = current_state_change_label
                single_change_point_record.loc[:, 'veh_id'] = current_veh_id
                single_change_point_record.loc[:, 'test_acc'] = current_acc
                single_change_point_record.loc[:, 'test_dec'] = '-'
                single_change_point_record.loc[:, 'test_jerk'] = current_absolute_jerk
                # for 1st example: 8 - (1 / 0.5) 
                single_change_point_record.loc[:, 'change_point_time_1'] = current_change_point_time - (current_acc / current_absolute_jerk)
                # 8
                single_change_point_record.loc[:, 'change_point_time_2'] = current_change_point_time
                # 0.5 * (6 + 8) = 7
                single_change_point_record.loc[:, 'mid_change_point_time'] = 0.5 * (single_change_point_record.loc[:, 'change_point_time_1'] + single_change_point_record.loc[:, 'change_point_time_2'])
                # 8
                single_change_point_record.loc[:, 'zero_acc_time'] = current_change_point_time
                # 0
                single_scenario_trj.loc[:, 'state_change_label'] = current_state_change_label
                # 0
                single_scenario_trj.loc[:, 'veh_id'] = current_veh_id
                
                # calculate the acceleration first
                # (8/0.1)
                change_point_step = round(current_change_point_time / time_resolution)
                # 80 - ((1/0.5)/0.1)
                acc_change_step = change_point_step - round((current_acc / current_absolute_jerk) / time_resolution)
                for t_step in range(total_time_step):
                    if t_step <= acc_change_step:
                        single_scenario_trj.loc[t_step, 'acceleration'] = current_acc
                    elif t_step < change_point_step:
                        single_scenario_trj.loc[t_step, 'acceleration'] = single_scenario_trj.loc[t_step - 1, 'acceleration'] - current_absolute_jerk * time_resolution
                    else:
                        single_scenario_trj.loc[t_step, 'acceleration'] = 0
                        
                # update speed and position
                for t_step in range(total_time_step):
                    
                    guass_noise = random.gauss(mu,sigma)
                    
                    single_scenario_trj.loc[t_step, 'local_time'] = t_step * time_resolution
                    if t_step == 0:
                        single_scenario_trj.loc[t_step, 'speed'] = guass_noise + current_starting_speed
                        single_scenario_trj.loc[t_step, 'position'] = 0
                    else:
                        previous_speed = single_scenario_trj.loc[t_step - 1, 'speed']
                        previous_position = single_scenario_trj.loc[t_step - 1, 'position']
                        single_scenario_trj.loc[t_step, 'speed'] = guass_noise + previous_speed + 0.5 * (single_scenario_trj.loc[t_step - 1, 'acceleration'] +single_scenario_trj.loc[t_step, 'acceleration']) * time_resolution
                        single_scenario_trj.loc[t_step, 'position'] = previous_position + 0.5 * (single_scenario_trj.loc[t_step, 'speed'] + previous_speed) * time_resolution
                all_state_change_point_record = all_state_change_point_record.append(single_change_point_record)
                all_synthetic_trj = all_synthetic_trj.append(single_scenario_trj)
                current_veh_id += 1

In [5]:
def A_D_trajectory_generation():
    global all_state_change_point_record
    global all_synthetic_trj
    
    current_state_change_label = 1
    current_starting_speed = 0
    current_veh_id = 200
    
    total_time_step, time_resolution, acceleration_range, acc_resolution, total_num_acc, deceleration_range, dec_resolution, total_num_dec, jerk_value_range, change_point_range, change_point_resolution, total_num_change_point = time_acc_dec_jerk_def()
    
    # random_deceleration = np.random.uniform(deceleration_range[0], deceleration_range[1], total_num_acc * total_num_change_point)
    for i in range(total_num_change_point):
        current_change_point_time = change_point_range[0] + i * change_point_resolution
        for j in range(total_num_acc):
            current_acc = acceleration_range[0] + acc_resolution * j
            for k in range(len(jerk_value_range)):
                current_absolute_jerk = jerk_value_range[k]
                print_str = 'Current generating ' + str(current_veh_id) + 'th vehicle synthetic trajectory '
                print(print_str)
                # the deceleration will be designated randomly
                # current_dec = random_deceleration[total_num_acc * i + j]
                acc_time = (current_change_point_time - (current_acc / current_absolute_jerk))
                current_dec = -(current_acc * acc_time) / (20 - acc_time)
                current_absolute_dec = abs(current_dec)
                single_scenario_trj = pd.DataFrame(np.zeros((total_time_step, 6)),
                                                   columns=['veh_id', 'local_time', 'position', 'speed', 'acceleration', 'state_change_label'])
                single_change_point_record = pd.DataFrame(np.zeros((1, 9)), columns=['veh_id', 'test_acc', 'test_dec', 'test_jerk',
                                                                   'change_point_time_1',
                                                                   'change_point_time_2',
                                                                   'zero_acc_time',
                                                                   'mid_change_point_time',
                                                                   'state_change_label'])
                single_change_point_record.loc[:, 'state_change_label'] = current_state_change_label
                single_change_point_record.loc[:, 'veh_id'] = current_veh_id
                single_change_point_record.loc[:, 'test_acc'] = current_acc
                single_change_point_record.loc[:, 'test_dec'] = current_dec
                single_change_point_record.loc[:, 'test_jerk'] = current_absolute_jerk
                single_change_point_record.loc[:, 'change_point_time_1'] = current_change_point_time - (current_acc / current_absolute_jerk)
                single_change_point_record.loc[:, 'change_point_time_2'] = current_change_point_time + (current_absolute_dec / current_absolute_jerk)
                single_change_point_record.loc[:, 'zero_acc_time'] = current_change_point_time
                single_change_point_record.loc[:, 'mid_change_point_time'] = 0.5 * (single_change_point_record.loc[:, 'change_point_time_1'] + single_change_point_record.loc[:, 'change_point_time_2'])
                single_scenario_trj.loc[:, 'state_change_label'] = current_state_change_label
                single_scenario_trj.loc[:, 'veh_id'] = current_veh_id
                # calculate the acceleration first
                change_point_step = int(current_change_point_time / time_resolution)
                acc_change_step = change_point_step - round((current_acc / current_absolute_jerk) / time_resolution)
                dec_change_step = change_point_step + round((current_absolute_dec / current_absolute_jerk) / time_resolution)
                for t_step in range(total_time_step):
                    if t_step <= acc_change_step:
                        single_scenario_trj.loc[t_step, 'acceleration'] = current_acc
                    elif t_step < dec_change_step:
                        single_scenario_trj.loc[t_step, 'acceleration'] = single_scenario_trj.loc[t_step - 1, 'acceleration'] - current_absolute_jerk * time_resolution
                    else:
                        single_scenario_trj.loc[t_step, 'acceleration'] = current_dec
                # update speed and position
                for t_step in range(total_time_step):
                    
                    guass_noise = random.gauss(mu,sigma)
                    
                    single_scenario_trj.loc[t_step, 'local_time'] = t_step * time_resolution
                    if t_step == 0:
                        single_scenario_trj.loc[t_step, 'speed'] = current_starting_speed+guass_noise
                        single_scenario_trj.loc[t_step, 'position'] = 0
                    else:
                        previous_speed = single_scenario_trj.loc[t_step - 1, 'speed']
                        previous_position = single_scenario_trj.loc[t_step - 1, 'position']
                        single_scenario_trj.loc[t_step, 'speed'] = guass_noise+previous_speed + 0.5 * (single_scenario_trj.loc[t_step - 1, 'acceleration'] +single_scenario_trj.loc[t_step, 'acceleration']) * time_resolution
                        single_scenario_trj.loc[t_step, 'position'] = previous_position + 0.5 * (single_scenario_trj.loc[t_step, 'speed'] + previous_speed) * time_resolution
                all_state_change_point_record = all_state_change_point_record.append(single_change_point_record)
                all_synthetic_trj = all_synthetic_trj.append(single_scenario_trj)
                current_veh_id += 1

In [6]:
def S_A_trajectory_generation():
    global all_state_change_point_record
    global all_synthetic_trj
    current_state_change_label = 2
    current_starting_speed = 0
    current_veh_id = 400
    total_time_step, time_resolution, acceleration_range, acc_resolution, total_num_acc, deceleration_range, dec_resolution, total_num_dec, jerk_value_range, change_point_range, change_point_resolution, total_num_change_point = time_acc_dec_jerk_def()
    
    for i in range(total_num_change_point):
        current_change_point_time = change_point_range[0] + i * change_point_resolution
        for j in range(total_num_acc):
            current_acc = acceleration_range[0] + acc_resolution * j
            for k in range(len(jerk_value_range)):
                current_absolute_jerk = jerk_value_range[k]
                print_str = 'Current generating ' + str(current_veh_id) + 'th vehicle synthetic trajectory '
                print(print_str)
                single_scenario_trj = pd.DataFrame(np.zeros((total_time_step, 6)), columns=['veh_id', 'local_time', 'position', 'speed', 'acceleration', 'state_change_label'])
                single_change_point_record = pd.DataFrame(np.zeros((1, 9)), columns=['veh_id', 'test_acc', 'test_dec', 'test_jerk',
                                                                   'change_point_time_1',
                                                                   'change_point_time_2',
                                                                   'zero_acc_time',
                                                                   'mid_change_point_time',
                                                                   'state_change_label'])
                single_change_point_record.loc[:, 'state_change_label'] = current_state_change_label
                single_change_point_record.loc[:, 'veh_id'] = current_veh_id
                single_change_point_record.loc[:, 'test_acc'] = current_acc
                single_change_point_record.loc[:, 'test_dec'] = '-'
                single_change_point_record.loc[:, 'test_jerk'] = current_absolute_jerk
                single_change_point_record.loc[:, 'change_point_time_1'] = current_change_point_time
                single_change_point_record.loc[:, 'change_point_time_2'] = current_change_point_time + (current_acc / current_absolute_jerk)
                single_change_point_record.loc[:, 'zero_acc_time'] = current_change_point_time
                single_change_point_record.loc[:, 'mid_change_point_time'] = 0.5 * (single_change_point_record.loc[:, 'change_point_time_1'] + single_change_point_record.loc[:, 'change_point_time_2'])
                single_scenario_trj.loc[:, 'state_change_label'] = current_state_change_label
                single_scenario_trj.loc[:, 'veh_id'] = current_veh_id
                # calculate the acceleration first
                change_point_step = round(current_change_point_time / time_resolution)
                acc_change_step = change_point_step + round((current_acc / current_absolute_jerk) / time_resolution)
                for t_step in range(total_time_step):
                    if t_step <= change_point_step:
                        single_scenario_trj.loc[t_step, 'acceleration'] = 0
                    elif t_step < acc_change_step:
                        single_scenario_trj.loc[t_step, 'acceleration'] = single_scenario_trj.loc[t_step - 1, 'acceleration'] + current_absolute_jerk * time_resolution
                    else:
                        single_scenario_trj.loc[t_step, 'acceleration'] = current_acc
                # update speed and position
                for t_step in range(total_time_step):
                    
                    guass_noise = random.gauss(mu,sigma)
                    
                    single_scenario_trj.loc[t_step, 'local_time'] = t_step * time_resolution
                    if t_step == 0:
                        single_scenario_trj.loc[t_step, 'speed'] = current_starting_speed+guass_noise
                        single_scenario_trj.loc[t_step, 'position'] = 0
                    else:
                        previous_speed = single_scenario_trj.loc[t_step - 1, 'speed']
                        previous_position = single_scenario_trj.loc[t_step - 1, 'position']
                        single_scenario_trj.loc[t_step, 'speed'] = guass_noise+previous_speed + 0.5 * (single_scenario_trj.loc[t_step - 1, 'acceleration'] +single_scenario_trj.loc[t_step, 'acceleration']) * time_resolution
                        single_scenario_trj.loc[t_step, 'position'] = previous_position + 0.5 * (single_scenario_trj.loc[t_step, 'speed'] + previous_speed) * time_resolution
                all_state_change_point_record = all_state_change_point_record.append(single_change_point_record)
                all_synthetic_trj = all_synthetic_trj.append(single_scenario_trj)
                current_veh_id += 1

In [7]:
def F_D_trajectory_generation():
    global all_state_change_point_record
    global all_synthetic_trj
    current_state_change_label = 3
    current_starting_speed = 36
    current_veh_id = 600
    total_time_step, time_resolution, acceleration_range, acc_resolution, total_num_acc, deceleration_range, dec_resolution, total_num_dec, jerk_value_range, change_point_range, change_point_resolution, total_num_change_point = time_acc_dec_jerk_def()
    
    for i in range(total_num_change_point):
        current_change_point_time = change_point_range[0] + i * change_point_resolution
        for j in range(total_num_dec):
            current_dec = deceleration_range[0] + dec_resolution * j
            absolute_dec = abs(current_dec)
            for k in range(len(jerk_value_range)):
                current_absolute_jerk = jerk_value_range[k]
                print_str = 'Current generating ' + str(current_veh_id) + 'th vehicle synthetic trajectory '
                print(print_str)
                single_scenario_trj = pd.DataFrame(np.zeros((total_time_step, 6)), columns=['veh_id', 'local_time', 'position', 'speed', 'acceleration', 'state_change_label'])
                single_change_point_record = pd.DataFrame(np.zeros((1, 9)), columns=['veh_id', 'test_acc', 'test_dec', 'test_jerk',
                                                                   'change_point_time_1',
                                                                   'change_point_time_2',
                                                                   'zero_acc_time',
                                                                   'mid_change_point_time',
                                                                   'state_change_label'])
                single_change_point_record.loc[:, 'state_change_label'] = current_state_change_label
                single_change_point_record.loc[:, 'veh_id'] = current_veh_id
                single_change_point_record.loc[:, 'test_acc'] = '-'
                single_change_point_record.loc[:, 'test_dec'] = current_dec
                single_change_point_record.loc[:, 'test_jerk'] = current_absolute_jerk
                single_change_point_record.loc[:, 'change_point_time_1'] = current_change_point_time
                single_change_point_record.loc[:, 'change_point_time_2'] = current_change_point_time + (absolute_dec / current_absolute_jerk)
                single_change_point_record.loc[:, 'zero_acc_time'] = current_change_point_time
                single_change_point_record.loc[:, 'mid_change_point_time'] = 0.5 * (single_change_point_record.loc[:, 'change_point_time_1'] + single_change_point_record.loc[:, 'change_point_time_2'])
                single_scenario_trj.loc[:, 'state_change_label'] = current_state_change_label
                single_scenario_trj.loc[:, 'veh_id'] = current_veh_id
                # calculate the acceleration first
                change_point_step = round(current_change_point_time / time_resolution)
                acc_change_step = change_point_step + round((absolute_dec / current_absolute_jerk) / time_resolution)
                for t_step in range(total_time_step):
                    if t_step <= change_point_step:
                        single_scenario_trj.loc[t_step, 'acceleration'] = 0
                    elif t_step < acc_change_step:
                        single_scenario_trj.loc[t_step, 'acceleration'] = single_scenario_trj.loc[t_step - 1, 'acceleration'] - current_absolute_jerk * time_resolution
                    else:
                        single_scenario_trj.loc[t_step, 'acceleration'] = current_dec
                # update speed and position
                for t_step in range(total_time_step):
                    
                    guass_noise = random.gauss(mu,sigma)
                    
                    single_scenario_trj.loc[t_step, 'local_time'] = t_step * time_resolution
                    if t_step == 0:
                        single_scenario_trj.loc[t_step, 'speed'] = current_starting_speed+guass_noise
                        single_scenario_trj.loc[t_step, 'position'] = 0
                    else:
                        previous_speed = single_scenario_trj.loc[t_step - 1, 'speed']
                        previous_position = single_scenario_trj.loc[t_step - 1, 'position']
                        single_scenario_trj.loc[t_step, 'speed'] = guass_noise+previous_speed + 0.5 * (single_scenario_trj.loc[t_step - 1, 'acceleration'] +single_scenario_trj.loc[t_step, 'acceleration']) * time_resolution
                        single_scenario_trj.loc[t_step, 'position'] = previous_position + 0.5 * (single_scenario_trj.loc[t_step, 'speed'] + previous_speed) * time_resolution
                all_state_change_point_record = all_state_change_point_record.append(single_change_point_record)
                all_synthetic_trj = all_synthetic_trj.append(single_scenario_trj)
                current_veh_id += 1

In [8]:
def D_S_trajectory_generation():
    global all_state_change_point_record
    global all_synthetic_trj
    current_state_change_label = 4
    # note that starting speed is set dynamically
    current_veh_id = 800
    
    total_time_step, time_resolution, acceleration_range, acc_resolution, total_num_acc, deceleration_range, dec_resolution, total_num_dec, jerk_value_range, change_point_range, change_point_resolution, total_num_change_point = time_acc_dec_jerk_def()
    
    for i in range(total_num_change_point):
        current_change_point_time = change_point_range[0] + i * change_point_resolution
        for j in range(total_num_dec):
            current_dec = deceleration_range[0] + dec_resolution * j
            absolute_dec = abs(current_dec)
            for k in range(len(jerk_value_range)):
                current_absolute_jerk = jerk_value_range[k]
                print_str = 'Current generating ' + str(current_veh_id) + 'th vehicle synthetic trajectory '
                print(print_str)
                current_starting_speed = (current_change_point_time - (absolute_dec / current_absolute_jerk)) * absolute_dec + 0.5 * absolute_dec * (absolute_dec / current_absolute_jerk)
                single_scenario_trj = pd.DataFrame(np.zeros((total_time_step, 6)), columns=['veh_id', 'local_time', 'position', 'speed', 'acceleration', 'state_change_label'])
                single_change_point_record = pd.DataFrame(np.zeros((1, 9)), columns=['veh_id', 'test_acc', 'test_dec', 'test_jerk',
                                                                   'change_point_time_1',
                                                                   'change_point_time_2',
                                                                   'zero_acc_time',
                                                                   'mid_change_point_time',
                                                                   'state_change_label'])
                single_change_point_record.loc[:, 'state_change_label'] = current_state_change_label
                single_change_point_record.loc[:, 'veh_id'] = current_veh_id
                single_change_point_record.loc[:, 'test_acc'] = '-'
                single_change_point_record.loc[:, 'test_dec'] = current_dec
                single_change_point_record.loc[:, 'test_jerk'] = current_absolute_jerk
                single_change_point_record.loc[:, 'change_point_time_1'] = current_change_point_time - (absolute_dec / current_absolute_jerk)
                single_change_point_record.loc[:, 'change_point_time_2'] = current_change_point_time
                single_change_point_record.loc[:, 'zero_acc_time'] = current_change_point_time
                single_change_point_record.loc[:, 'mid_change_point_time'] = 0.5 * (single_change_point_record.loc[:, 'change_point_time_1'] + single_change_point_record.loc[:, 'change_point_time_2'])
                single_scenario_trj.loc[:, 'state_change_label'] = current_state_change_label
                single_scenario_trj.loc[:, 'veh_id'] = current_veh_id
                # calculate the acceleration first
                change_point_step = round(current_change_point_time / time_resolution)
                acc_change_step = change_point_step - round((absolute_dec / current_absolute_jerk) / time_resolution)
                for t_step in range(total_time_step):
                    if t_step <= acc_change_step:
                        single_scenario_trj.loc[t_step, 'acceleration'] = current_dec
                    elif t_step < change_point_step:
                        single_scenario_trj.loc[t_step, 'acceleration'] = single_scenario_trj.loc[t_step - 1, 'acceleration'] + current_absolute_jerk * time_resolution
                    else:
                        single_scenario_trj.loc[t_step, 'acceleration'] = 0
                # update speed and position
                for t_step in range(total_time_step):
                    
                    guass_noise = random.gauss(mu,sigma)
                    
                    single_scenario_trj.loc[t_step, 'local_time'] = t_step * time_resolution
                    if t_step == 0:
                        single_scenario_trj.loc[t_step, 'speed'] = current_starting_speed+guass_noise
                        single_scenario_trj.loc[t_step, 'position'] = 0
                    elif t_step >= change_point_step:
                        previous_speed = single_scenario_trj.loc[t_step - 1, 'speed']
                        previous_position = single_scenario_trj.loc[t_step - 1, 'position']
                        single_scenario_trj.loc[t_step, 'speed'] = 0 +guass_noise
                        single_scenario_trj.loc[t_step, 'position'] = previous_position + 0.5 * (single_scenario_trj.loc[t_step, 'speed'] + previous_speed) * time_resolution
                    else:
                        previous_speed = single_scenario_trj.loc[t_step - 1, 'speed']
                        previous_position = single_scenario_trj.loc[t_step - 1, 'position']
                        single_scenario_trj.loc[t_step, 'speed'] = guass_noise+previous_speed + 0.5 * (single_scenario_trj.loc[t_step - 1, 'acceleration'] +single_scenario_trj.loc[t_step, 'acceleration']) * time_resolution
                        if single_scenario_trj.loc[t_step, 'speed'] < 0:
                            # modify the change point if speed drop to 0 in advance
                            single_scenario_trj.loc[t_step, 'speed'] = 0+guass_noise
                            single_scenario_trj.loc[t_step, 'acceleration'] = 0
                            single_change_point_record.loc[:, 'change_point_time_2'] = t_step * time_resolution
                        single_scenario_trj.loc[t_step, 'position'] = previous_position + 0.5 * (single_scenario_trj.loc[t_step, 'speed'] + previous_speed) * time_resolution
                all_state_change_point_record = all_state_change_point_record.append(single_change_point_record)
                all_synthetic_trj = all_synthetic_trj.append(single_scenario_trj)
                current_veh_id += 1

In [9]:
synthetic_trj_generation_label = 1
wavelet_scale_sensitivity_analysis_label = 0
wavelet_scale_sensitivity_video_generation = 0
local_maxima_lines_test_label = 0

In [10]:
if synthetic_trj_generation_label:
    all_state_change_point_record = pd.DataFrame()
    all_synthetic_trj = pd.DataFrame()
    #wavelet_detection_synthetic_trajectory()
    A_F_trajectory_generation()
    A_D_trajectory_generation()
    S_A_trajectory_generation()
    F_D_trajectory_generation()
    D_S_trajectory_generation()
    all_state_change_point_record.to_csv('./data_save/all_state_change_point_record.csv')
    all_synthetic_trj.to_csv('./data_save/all_synthetic_trj.csv')

Current generating 0th vehicle synthetic trajectory 
Current generating 1th vehicle synthetic trajectory 
Current generating 2th vehicle synthetic trajectory 
Current generating 3th vehicle synthetic trajectory 
Current generating 4th vehicle synthetic trajectory 
Current generating 5th vehicle synthetic trajectory 
Current generating 6th vehicle synthetic trajectory 
Current generating 7th vehicle synthetic trajectory 
Current generating 8th vehicle synthetic trajectory 
Current generating 9th vehicle synthetic trajectory 
Current generating 10th vehicle synthetic trajectory 
Current generating 11th vehicle synthetic trajectory 
Current generating 12th vehicle synthetic trajectory 
Current generating 13th vehicle synthetic trajectory 
Current generating 14th vehicle synthetic trajectory 
Current generating 15th vehicle synthetic trajectory 
Current generating 16th vehicle synthetic trajectory 
Current generating 17th vehicle synthetic trajectory 
Current generating 18th vehicle synthe

Current generating 152th vehicle synthetic trajectory 
Current generating 153th vehicle synthetic trajectory 
Current generating 154th vehicle synthetic trajectory 
Current generating 155th vehicle synthetic trajectory 
Current generating 156th vehicle synthetic trajectory 
Current generating 157th vehicle synthetic trajectory 
Current generating 158th vehicle synthetic trajectory 
Current generating 159th vehicle synthetic trajectory 
Current generating 160th vehicle synthetic trajectory 
Current generating 161th vehicle synthetic trajectory 
Current generating 162th vehicle synthetic trajectory 
Current generating 163th vehicle synthetic trajectory 
Current generating 164th vehicle synthetic trajectory 
Current generating 165th vehicle synthetic trajectory 
Current generating 166th vehicle synthetic trajectory 
Current generating 167th vehicle synthetic trajectory 
Current generating 168th vehicle synthetic trajectory 
Current generating 169th vehicle synthetic trajectory 
Current ge

Current generating 302th vehicle synthetic trajectory 
Current generating 303th vehicle synthetic trajectory 
Current generating 304th vehicle synthetic trajectory 
Current generating 305th vehicle synthetic trajectory 
Current generating 306th vehicle synthetic trajectory 
Current generating 307th vehicle synthetic trajectory 
Current generating 308th vehicle synthetic trajectory 
Current generating 309th vehicle synthetic trajectory 
Current generating 310th vehicle synthetic trajectory 
Current generating 311th vehicle synthetic trajectory 
Current generating 312th vehicle synthetic trajectory 
Current generating 313th vehicle synthetic trajectory 
Current generating 314th vehicle synthetic trajectory 
Current generating 315th vehicle synthetic trajectory 
Current generating 316th vehicle synthetic trajectory 
Current generating 317th vehicle synthetic trajectory 
Current generating 318th vehicle synthetic trajectory 
Current generating 319th vehicle synthetic trajectory 
Current ge

Current generating 452th vehicle synthetic trajectory 
Current generating 453th vehicle synthetic trajectory 
Current generating 454th vehicle synthetic trajectory 
Current generating 455th vehicle synthetic trajectory 
Current generating 456th vehicle synthetic trajectory 
Current generating 457th vehicle synthetic trajectory 
Current generating 458th vehicle synthetic trajectory 
Current generating 459th vehicle synthetic trajectory 
Current generating 460th vehicle synthetic trajectory 
Current generating 461th vehicle synthetic trajectory 
Current generating 462th vehicle synthetic trajectory 
Current generating 463th vehicle synthetic trajectory 
Current generating 464th vehicle synthetic trajectory 
Current generating 465th vehicle synthetic trajectory 
Current generating 466th vehicle synthetic trajectory 
Current generating 467th vehicle synthetic trajectory 
Current generating 468th vehicle synthetic trajectory 
Current generating 469th vehicle synthetic trajectory 
Current ge

Current generating 602th vehicle synthetic trajectory 
Current generating 603th vehicle synthetic trajectory 
Current generating 604th vehicle synthetic trajectory 
Current generating 605th vehicle synthetic trajectory 
Current generating 606th vehicle synthetic trajectory 
Current generating 607th vehicle synthetic trajectory 
Current generating 608th vehicle synthetic trajectory 
Current generating 609th vehicle synthetic trajectory 
Current generating 610th vehicle synthetic trajectory 
Current generating 611th vehicle synthetic trajectory 
Current generating 612th vehicle synthetic trajectory 
Current generating 613th vehicle synthetic trajectory 
Current generating 614th vehicle synthetic trajectory 
Current generating 615th vehicle synthetic trajectory 
Current generating 616th vehicle synthetic trajectory 
Current generating 617th vehicle synthetic trajectory 
Current generating 618th vehicle synthetic trajectory 
Current generating 619th vehicle synthetic trajectory 
Current ge

Current generating 751th vehicle synthetic trajectory 
Current generating 752th vehicle synthetic trajectory 
Current generating 753th vehicle synthetic trajectory 
Current generating 754th vehicle synthetic trajectory 
Current generating 755th vehicle synthetic trajectory 
Current generating 756th vehicle synthetic trajectory 
Current generating 757th vehicle synthetic trajectory 
Current generating 758th vehicle synthetic trajectory 
Current generating 759th vehicle synthetic trajectory 
Current generating 760th vehicle synthetic trajectory 
Current generating 761th vehicle synthetic trajectory 
Current generating 762th vehicle synthetic trajectory 
Current generating 763th vehicle synthetic trajectory 
Current generating 764th vehicle synthetic trajectory 
Current generating 765th vehicle synthetic trajectory 
Current generating 766th vehicle synthetic trajectory 
Current generating 767th vehicle synthetic trajectory 
Current generating 768th vehicle synthetic trajectory 
Current ge

Current generating 900th vehicle synthetic trajectory 
Current generating 901th vehicle synthetic trajectory 
Current generating 902th vehicle synthetic trajectory 
Current generating 903th vehicle synthetic trajectory 
Current generating 904th vehicle synthetic trajectory 
Current generating 905th vehicle synthetic trajectory 
Current generating 906th vehicle synthetic trajectory 
Current generating 907th vehicle synthetic trajectory 
Current generating 908th vehicle synthetic trajectory 
Current generating 909th vehicle synthetic trajectory 
Current generating 910th vehicle synthetic trajectory 
Current generating 911th vehicle synthetic trajectory 
Current generating 912th vehicle synthetic trajectory 
Current generating 913th vehicle synthetic trajectory 
Current generating 914th vehicle synthetic trajectory 
Current generating 915th vehicle synthetic trajectory 
Current generating 916th vehicle synthetic trajectory 
Current generating 917th vehicle synthetic trajectory 
Current ge